In [5]:
import random
import os
import numpy as np
from midiutil import MIDIFile
import pretty_midi
import soundfile as sf
import sounddevice as sd
from midi2audio import FluidSynth


def generate_chord_progression():
    progressions = [
        ["C", "G", "Am", "F"],
        ["C", "Am", "F", "G"],
        ["G", "Em", "C", "D"],
        ["Am", "F", "C", "G"],
    ]
    return random.choice(progressions)


def generate_midi(filename, chord_progression, instrument_program):
    midi = MIDIFile(1)
    track = 0
    time = 0
    midi.addTrackName(track, time, "Sample Track")
    midi.addTempo(track, time, 120)

    channel = 0
    volume = 100
    duration = 1  # 1 beat per chord

    # Set the instrument
    midi.addProgramChange(track, channel, time, instrument_program)

    chord_to_note = {"C": 60, "G": 67, "Am": 69, "F": 65, "Em": 64, "D": 62}

    for chord in chord_progression:
        base_note = chord_to_note[chord]
        midi.addNote(track, channel, base_note, time, duration, volume)
        midi.addNote(
            track,
            channel,
            base_note + (3 if "m" in chord else 4),
            time,
            duration,
            volume,
        )
        midi.addNote(track, channel, base_note + 7, time, duration, volume)
        time += 1

    with open(filename, "wb") as output_file:
        midi.writeFile(output_file)


def midi_to_wav(midi_file, wav_file):
    FluidSynth().midi_to_audio(midi_file, wav_file)


def play_audio(file_path):
    data, fs = sf.read(file_path, dtype="float32")
    sd.play(data, fs)
    sd.wait()


def check_instrument_sounds():
    instruments = [
        ("Acoustic Grand Piano", 0),
        ("Electric Guitar (clean)", 27),
        ("Violin", 40),
        ("Flute", 73),
    ]
    chord_progression = generate_chord_progression()

    for instrument_name, program_number in instruments:
        print(f"\nGenerating and playing sound for {instrument_name}...")
        midi_file = f'temp_{instrument_name.replace(" ", "_")}.mid'
        wav_file = f'temp_{instrument_name.replace(" ", "_")}.wav'

        generate_midi(midi_file, chord_progression, program_number)
        midi_to_wav(midi_file, wav_file)

        print(f"Playing {instrument_name} sound...")
        play_audio(wav_file)

        os.remove(midi_file)
        os.remove(wav_file)

        input("Press Enter to continue to the next instrument...")


# Run the instrument sound check
check_instrument_sounds()


Generating and playing sound for Acoustic Grand Piano...
FluidSynth runtime version 2.3.6
Copyright (C) 2000-2024 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file 'temp_Acoustic_Grand_Piano.wav'..
Playing Acoustic Grand Piano sound...


fluidsynth: panic: An error occurred while reading from stdin.
fluidsynth: panic: An error occurred while reading from stdin.



Generating and playing sound for Electric Guitar (clean)...
FluidSynth runtime version 2.3.6
Copyright (C) 2000-2024 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file 'temp_Electric_Guitar_(clean).wav'..
Playing Electric Guitar (clean) sound...


fluidsynth: panic: An error occurred while reading from stdin.
fluidsynth: panic: An error occurred while reading from stdin.


KeyboardInterrupt: 